# **OTTO Recommender**

**Integrantes:**
- Jakelin Daiana Correa Palacio
- Daniela Guardia Cuervo
- Dayra Gabriela Chamorro Revelo
- Fredy Alberto Orozco Loaiza (Además presentó certificado de participación en Datatón Bancolombia)
- Cristian Camilo Henao Rojas

# Introducción

El objetivo de este trabajo es presentar una solución al problema descrito en la competencia de Kaggle [OTTO – Multi-Objective Recommender System](https://www.kaggle.com/competitions/otto-recommender-system/overview). El objetivo de esta competencia es predecir los clics de comercio electrónico, las adiciones al carrito y los pedidos mediante un sistema de recomendación multiobjetivo basado en eventos anteriores en una sesión de usuario.



## Importe de librerías

In [1]:
# importamos libreías a utilizar
import pandas as pd
import numpy as np

### Datos


El dataset contiene información acerca de las sesiones de ususarios y los eventos registrados durante esa session.
Los campos son: 
- session: el id de la sesión.
- **aid:** id del artículo seleccionado.
- **ts:** el tiempo en el que ocurrió el evento (Tiempo Unix: cantidad de segundos transcurridos desde la medianoche UTC del 1 de enero de 1970)
- **type:** tipo de evento con tres valores: 0 --> le dio click, 1 --> lo agregó al carrito, 2 --> lo ordenó.

Se usa un [dataset optimizado](https://www.kaggle.com/datasets/radek1/otto-full-optimized-memory-footprint), para ahorrar el consumo de ram en el lab.

In [31]:
# leemos los datasets de entrenamiento y testeo
df_train = pd.read_parquet('train.parquet')
df_test = pd.read_parquet('test.parquet')

In [32]:
# observamos el tamaño del dataset de entrenamiento
df_train.shape

(216716096, 4)

In [33]:
# observamos el tamaño del dataset de testeo
df_test.shape

(6928123, 4)

In [34]:
#df_train = df_train.sample(1000000, random_state= 42)
#df_test = df_test.sample(100000, random_state= 42)

In [35]:
# observamos las primeras filas del dataset de entrenamiento
df_train.head()

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0


In [36]:
# observamos las primeras filas del dataset de testeo
df_test.head()

,session,aid,ts,type
0,12899779,59625,1661724000,0
1,12899780,1142000,1661724000,0
2,12899780,582732,1661724058,0
3,12899780,973453,1661724109,0
4,12899780,736515,1661724136,0


In [37]:
# observamos la cantidad de eventos por sesión en el dataset de prueba
df_test.value_counts('session')

session
14086824    458
13352358    433
13242700    421
13234219    407
13346726    406
           ... 
13675903      1
13675907      1
13675908      1
13675909      1
14571581      1
Length: 1671803, dtype: int64

In [38]:
# observamos la cantidad de eventos por sesión en el dataset de entrenamiento
df_train.value_counts('session')

session
6265682     500
4110619     500
10776795    499
1870739     499
2473055     498
           ... 
10731401      2
6311756       2
10731403      2
10731405      2
12899778      2
Length: 12899779, dtype: int64

In [39]:
# observamos los posibles tipos de evento 
df_train.type.unique()

array([0, 1, 2], dtype=uint8)

Se presencian 3 tipos de evento, donde 0 indica que el usuario dió click sobre un producto, 1 indica que agregó un producto al carrito y 2 indica que compró un producto.

In [40]:
# definimos funciones a utilizar

# función que toma los últimos 20 productos del dataframe y los retorna en una lista
def last_20(df):
  return list(df.tail(20)['aid'])

# función que une los productos según el evento con el top 20 de productos y toma sólo los primeros, además los convierte a str y los une mediante espacio
def refill_top_20(labels):
  return ' '.join(map(str, (labels + products_top20)[:20]))

In [41]:
# obtenemos la cantidad de sesiones en el dataset de entrenamiento
session_count = len(pd.unique(df_train['session']))
# obtenemos la cantidad de productos registrados en el dataset de entrenamiento
items_count = len(pd.unique(df_train['aid']))
# obtenemos la cantidad de eventos en el dataset de entrenamiento
events_count = df_train['type'].count()
# obtenemos la cantidad de eventos según su tipo en el dataset de entrenamiento
events = df_train['type'].value_counts()

In [42]:
# observamos resumen de datos del dataset de entrenamiento
data = [[session_count, items_count, events_count, events[0], events[1], events[2]]]
cols = ['#sessions', '#items', '#events', '#clicks', '#carts', '#orders']
row = [['train']]
summary = pd.DataFrame(data, columns=cols, index=row)
summary

,#sessions,#items,#events,#clicks,#carts,#orders
train,12899779,1855603,216716096,194720954,16896191,5098951


In [43]:
print('El ' + str(events[0]/events_count*100) + '% de los eventos son "clicks"')
print('El ' + str(events[1]/events_count*100) + '% de los eventos son "agregar al carrito"')
print('El ' + str(events[2]/events_count*100) + '% de los eventos son "ordenado"')

El 89.85071141185563% de los eventos son "clicks"
El 7.796463350834817% de los eventos son "agregar al carrito"
El 2.3528252373095535% de los eventos son "ordenado"


In [44]:
# obtenemos la cantidad de veces que se realizó un evento sobre un producto, los ordenamos de mayor a menor y obtenemos el top 20
products_top20 = df_train.groupby('aid').session.count()
products_top20 = products_top20.sort_values(ascending=False)
products_top20 = list(products_top20.head(20))
print(products_top20)

[129004, 126836, 118524, 113279, 105091, 91325, 90244, 84657, 80197, 79872, 78203, 76988, 76737, 74027, 74026, 71732, 68329, 66962, 65124, 64900]


In [45]:
# observamos el top 20 de los productos según el evento 'click'
top20_clicks = df_train[df_train.type == 0].groupby('aid').ts.count().sort_values(ascending=False).head(20)
print(top20_clicks)

aid
1460571    121287
108125     114456
29735      101148
485256      97154
1733943     91395
184976      85122
832192      81127
1502122     73805
554660      72161
1603001     68519
986164      67791
166037      67357
322370      66729
1236775     66076
231487      64994
959208      63034
332654      62617
1196256     62221
95488       62023
620545      60816
Name: ts, dtype: int64


In [46]:
# observamos el top 20 de los productos según el evento 'cart'
top20_carts = df_train[df_train.type == 1].groupby('aid').ts.count().sort_values(ascending=False).head(20)
print(top20_carts)

aid
485256     29682
152547     18536
33343      14726
166037     13476
1733943    10654
231487     10393
29735      10124
1022566     9338
832192      8626
544144      7407
554660      7336
322370      7298
1562705     7128
986164      7005
1083665     6734
332654      6679
1629608     6659
1236775     6224
756588      6166
613493      6111
Name: ts, dtype: int64


In [47]:
# observamos el top 20 de los productos según el evento 'order'
top20_orders = df_train[df_train.type == 2].groupby('aid').ts.count().sort_values(ascending=False).head(20)
print(top20_orders)

aid
231487     4485
166037     3824
1733943    3042
1445562    2998
1022566    2788
801774     2710
1629608    2628
756588     2593
332654     2436
1603001    2353
409620     2329
1257293    2273
1125638    2203
986164     2192
1083665    2174
450505     2137
544144     2096
1025795    2071
125278     2047
29735      2007
Name: ts, dtype: int64


In [48]:
#Buscamos el top20 para cada tipo de evento

top20_clicks = list(top20_clicks.index)
top20_clicks_str = " ".join(map(str,top20_clicks))

top20_carts = list(top20_carts.index)
top20_carts_str = " ".join(map(str,top20_carts))

top20_orders = list(top20_orders.index)
top20_orders_str = " ".join(map(str,top20_orders))

top20_clicks_str

'1460571 108125 29735 485256 1733943 184976 832192 1502122 554660 1603001 986164 166037 322370 1236775 231487 959208 332654 1196256 95488 620545'

In [53]:
# Agrupamos los productos por sesion tomando los ultimos 20 registros

pred_df = df_test.sort_values(["session", "type", "ts"])
pred_df = pred_df.groupby(["session"]).apply(lambda x: last_20(x)).reset_index()

pred_df.head()

,session,0
0,12899779,[59625]
1,12899780,"[1142000, 582732, 973453, 736515, 1142000]"
2,12899781,"[141736, 199008, 57315, 194067, 199008, 199008..."
3,12899782,"[476063, 779477, 975116, 595994, 1344773, 1711..."
4,12899783,"[255297, 1114789, 255297, 300127, 198385, 3001..."


In [58]:
# Para el envio debemos tener el dataframe con las columnas como 'session_type' y 'labels' 
# Se le crea un sufijo a cada session, según tipo de evento.

clicks_pred = pred_df.copy()
clicks_pred['session'] = clicks_pred['session'].astype(str) + '_clicks'

carts_pred = pred_df.copy()
carts_pred['session'] = carts_pred['session'].astype(str) + '_carts'

orders_pred = pred_df.copy()
orders_pred['session'] = orders_pred['session'].astype(str) + '_orders'

clicks_pred.rename(columns = {0:'labels'}, inplace = True)
carts_pred.rename(columns = {0:'labels'}, inplace = True)
orders_pred.rename(columns = {0:'labels'}, inplace = True)

clicks_pred

,session,labels
0,12899779_clicks,[59625]
1,12899780_clicks,"[1142000, 582732, 973453, 736515, 1142000]"
2,12899781_clicks,"[141736, 199008, 57315, 194067, 199008, 199008..."
3,12899782_clicks,"[476063, 779477, 975116, 595994, 1344773, 1711..."
4,12899783_clicks,"[255297, 1114789, 255297, 300127, 198385, 3001..."
...,...,...
1671798,14571577_clicks,[1141710]
1671799,14571578_clicks,[519105]
1671800,14571579_clicks,[739876]
1671801,14571580_clicks,[202353]


In [60]:
#Aquellas sesiones con menos de 20 productos vistos, se rellenan 20 productos con los mas populares 
clicks_pred['labels'] = clicks_pred['labels'].apply(lambda x: refill_top_20(x))
carts_pred['labels'] = carts_pred['labels'].apply(lambda x: refill_top_20(x))
orders_pred['labels'] = orders_pred['labels'].apply(lambda x: refill_top_20(x))

clicks_pred

,session,labels
0,12899779_clicks,59625 129004 126836 118524 113279 105091 91325...
1,12899780_clicks,1142000 582732 973453 736515 1142000 129004 12...
2,12899781_clicks,141736 199008 57315 194067 199008 199008 19900...
3,12899782_clicks,476063 779477 975116 595994 1344773 1711180 12...
4,12899783_clicks,255297 1114789 255297 300127 198385 300127 172...
...,...,...
1671798,14571577_clicks,1141710 129004 126836 118524 113279 105091 913...
1671799,14571578_clicks,519105 129004 126836 118524 113279 105091 9132...
1671800,14571579_clicks,739876 129004 126836 118524 113279 105091 9132...
1671801,14571580_clicks,202353 129004 126836 118524 113279 105091 9132...


In [61]:
# Se prepara el dataset para cargarse a la plataforma
pred_df = pd.concat([clicks_pred, orders_pred, carts_pred])
pred_df.columns = ['session_type', 'labels']
pred_df

,session_type,labels
0,12899779_clicks,59625 129004 126836 118524 113279 105091 91325...
1,12899780_clicks,1142000 582732 973453 736515 1142000 129004 12...
2,12899781_clicks,141736 199008 57315 194067 199008 199008 19900...
3,12899782_clicks,476063 779477 975116 595994 1344773 1711180 12...
4,12899783_clicks,255297 1114789 255297 300127 198385 300127 172...
...,...,...
1671798,14571577_carts,1141710 129004 126836 118524 113279 105091 913...
1671799,14571578_carts,519105 129004 126836 118524 113279 105091 9132...
1671800,14571579_carts,739876 129004 126836 118524 113279 105091 9132...
1671801,14571580_carts,202353 129004 126836 118524 113279 105091 9132...


In [62]:
pred_df = pred_df.sort_values(by='session_type').reset_index()
pred_df = pred_df.drop('index', axis=1)
pred_df

,session_type,labels
0,12899779_carts,59625 129004 126836 118524 113279 105091 91325...
1,12899779_clicks,59625 129004 126836 118524 113279 105091 91325...
2,12899779_orders,59625 129004 126836 118524 113279 105091 91325...
3,12899780_carts,1142000 582732 973453 736515 1142000 129004 12...
4,12899780_clicks,1142000 582732 973453 736515 1142000 129004 12...
...,...,...
5015404,14571580_clicks,202353 129004 126836 118524 113279 105091 9132...
5015405,14571580_orders,202353 129004 126836 118524 113279 105091 9132...
5015406,14571581_carts,1100210 129004 126836 118524 113279 105091 913...
5015407,14571581_clicks,1100210 129004 126836 118524 113279 105091 913...


In [63]:
#Este csv es el que se sube a la plataforma.
pred_df.to_csv("submission.csv", index=False)